In [1]:
# 1. DATASET PREPARATION

#Loading Data
import pandas as pd


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


import pandas as pd

file_path = r"C:\Users\adity\Desktop\NLP\amazon_reviews_us_Office_Products_v1_00.tsv"
data = pd.read_csv(file_path, sep='\t', on_bad_lines="skip")
print (data)

C:\Users\adity\AppData\Local\Temp\ipykernel_1820\3622204569.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', on_bad_lines="skip")


        marketplace  customer_id       review_id  product_id  product_parent  \
0                US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1                US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2                US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3                US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4                US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   
...             ...          ...             ...         ...             ...   
2640249          US     53005790   RLI7EI10S7SN0  B00000DM9M       223408988   
2640250          US     52188548  R1F3SRK9MHE6A3  B00000DM9M       223408988   
2640251          US     52090046  R23V0C4NRJL8EM  0807865001       307284585   
2640252          US     52503173  R13ZAE1ATEUC1T  1572313188       870359649   
2640253          US     52585611   RE8J5O2GY04NN  1572313188       870359649   

                                       

In [3]:
data = data[['review_body', 'star_rating']]

In [4]:
data = data[data['star_rating'] !=3]

In [5]:
#Removing rows with non-numeric 'star_rating' values: 
data = data[pd.to_numeric(data['star_rating'], errors='coerce').notnull()]

#Converting 'star_rating' to integers: 
data['star_rating'] = data['star_rating'].astype(int)

#Function:
data['Sentiment'] = data['star_rating'].apply(lambda rating : +1 if rating > 3 else 0)


In [6]:
print (data)

                                               review_body  star_rating  \
0                                           Great product.            5   
1        What's to say about this commodity item except...            5   
2          Haven't used yet, but I am sure I will like it.            5   
3        Although this was labeled as &#34;new&#34; the...            1   
4                          Gorgeous colors and easy to use            4   
...                                                    ...          ...   
2640249  I can't live anymore whithout my Palm III. But...            4   
2640250  Although the Palm Pilot is thin and compact it...            4   
2640251  This book had a lot of great content without b...            4   
2640252  I am teaching a course in Excel and am using t...            5   
2640253  A very comprehensive layout of exactly how Vis...            5   

         Sentiment  
0                1  
1                1  
2                1  
3              

In [7]:
#Printing no. of reviews for each sentiment: 
print('Number of positive reviews:', len(data[data['Sentiment'] == 1]))
print('Number of negative reviews:', len(data[data['Sentiment'] == 0]))

Number of positive reviews: 2001183
Number of negative reviews: 459183


In [8]:
# Selecting 100,000 positive reviews and 100,000 negative reviews: 
data_pos = data[data['Sentiment'] == 1].sample(n=100000)
data_neg = data[data['Sentiment'] == 0].sample(n=100000)

In [9]:
# Concatenating the reviews:
data2 = pd.concat([data_pos, data_neg])
print(data2)

                                               review_body  star_rating  \
1919276  I'm in school for accounting and use a lot of ...            4   
726405                         Thank you it's amazing item            5   
1246962                      Great price, is working well.            5   
2393875  This screen protector is a perfect cut to fit ...            5   
396871   Although this is my first photo printer, I've ...            5   
...                                                    ...          ...   
1497376  I hoped this would produce some noise you coul...            1   
1932136  the ink printed maybe 3 pages before it ran ou...            1   
1299488                                     could not hear            1   
736985                                 was not compatible.            1   
177045   I feel somewhat ripped off. I bought the dark ...            2   

         Sentiment  
1919276          1  
726405           1  
1246962          1  
2393875        

In [10]:
train_data, test_data = train_test_split(data2, test_size=0.2, random_state=42)

In [11]:
#Splitting into training and testing sets: 
print('Number of reviews in the training set:', len(train_data))
print('Number of reviews in the testing set:', len(test_data))

Number of reviews in the training set: 160000
Number of reviews in the testing set: 40000


In [12]:
# 2. Data Cleaning

import re
def clean_text(text):
    
    text = str(text)

     # Convert text to lower case: 
    text = text.lower()

     # Remove HTML tags: 
    text = re.sub(r'<.*?>', '', text)

    # Remove URLs: 
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

     # Remove non-alphabetical characters: 
    text = re.sub(r'[^a-zA-Z\s]', '', text)

     # Remove extra spaces:
    text = re.sub(r'\s+', ' ', text).strip()


    return text

#Apply Cleaning:
data2['cleaned_review'] = data2['review_body'].apply(clean_text)



In [13]:
#  Average review length before cleaning

data2['review_length_before'] = data2['review_body'].astype(str).apply(len)
avg_length_before = data2['review_length_before'].mean()

# Apply the cleaning function
data2['cleaned_review'] = data2['review_body'].apply(clean_text)

#  Average review length after cleaning
data2['review_length_after'] = data2['cleaned_review'].apply(len)
avg_length_after = data2['review_length_after'].mean()

print("Average length before cleaning:", avg_length_before)
print("Average length after cleaning:", avg_length_after)


Average length before cleaning: 318.26275
Average length after cleaning: 300.41638


In [14]:
# 3. PRE-PROCESSING

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

#Defining Stop words and Lemmatizer: 
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

def preprocess_reviews(reviews):
    
    #Tokenization:  
    reviews = reviews.str.split()
    
   #Removing stop words and perform lemmatization : 
    reviews = reviews.apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in stop_words])
    
   #Joining back into single string:
    reviews = reviews.str.join(' ')
    
    return reviews

# Print 3 sample reviews before preprocessing
print('Before preprocessing:')
print(data2['review_body'].head(3))

data2['review_body'] = data2['review_body'].astype(str)

#  Average length of the reviews before preprocessing
average_length_before = data2['review_body'].apply(len).mean()
print('Average length before preprocessing:', average_length_before)

data2['review_body'] = preprocess_reviews(data2['review_body'])

# Print 3 sample reviews after preprocessing
print('After preprocessing:')
print(data2['review_body'].head(3))

# Average length of the reviews after preprocessing
average_length_after = data2['review_body'].apply(len).mean()
print('Average length after preprocessing:', average_length_after)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Before preprocessing:
1919276    I'm in school for accounting and use a lot of ...
726405                           Thank you it's amazing item
1246962                        Great price, is working well.
Name: review_body, dtype: object
Average length before preprocessing: 318.26275
After preprocessing:
1919276    I'm school accounting use lot lead. This far b...
726405                                    Thank amazing item
1246962                           Great price, working well.
Name: review_body, dtype: object
Average length after preprocessing: 221.881355


In [15]:
# 4. TF-IDF FEATURE EXTRACTION

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize
vectorizer = TfidfVectorizer()

#Transform the reviews into TF-IDF features: 
tfidf_features = vectorizer.fit_transform(data2['review_body'])

# Print the shape of the TF-IDF features:
print('Shape of TF-IDF features:', tfidf_features.shape)

Shape of TF-IDF features: (200000, 72739)


In [16]:
# 5. PERCEPTRON MODEL

from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets: 
X_train, X_test, y_train, y_test = train_test_split(tfidf_features, data2['Sentiment'], test_size=0.2, random_state=42)

perceptron = Perceptron()

# Training the model:
perceptron.fit(X_train, y_train)

# Using on training and testing data: 
y_train_pred = perceptron.predict(X_train)
y_test_pred = perceptron.predict(X_test)


train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
train_precision = precision_score(y_train, y_train_pred)
test_precision = precision_score(y_test, y_test_pred)
train_recall = recall_score(y_train, y_train_pred)
test_recall = recall_score(y_test, y_test_pred)
train_f1 = f1_score(y_train, y_train_pred)
test_f1 = f1_score(y_test, y_test_pred)

# Print:
print("Perceptron Training Metrics:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1 Score:", train_f1)

print("\nPerceptron Testing Metrics:")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1 Score:", test_f1)


Perceptron Training Metrics:
Accuracy: 0.9076625
Precision: 0.891211783191201
Recall: 0.9287062382041571
F1 Score: 0.9095727751254744

Perceptron Testing Metrics:
Accuracy: 0.855725
Precision: 0.842006540977299
Recall: 0.8756564797679187
F1 Score: 0.85850190020841


In [17]:
# 6. lOGISTIC REGRESSION AND MULTINOMIAL NAIVE BAYES

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB


log_reg_model = LogisticRegression(max_iter=1000)
nb_model = MultinomialNB()


log_reg_model.fit(X_train, y_train)
nb_model.fit(X_train, y_train)


models = {'Logistic Regression': log_reg_model, 'Naive Bayes': nb_model}

for name, model in models.items():
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print(f"{name} Training Metrics:")
    print("Accuracy:", accuracy_score(y_train, y_train_pred))
    print("Precision:", precision_score(y_train, y_train_pred))
    print("Recall:", recall_score(y_train, y_train_pred))
    print("F1 Score:", f1_score(y_train, y_train_pred))

    print(f"\n{name} Testing Metrics:")
    print("Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Precision:", precision_score(y_test, y_test_pred))
    print("Recall:", recall_score(y_test, y_test_pred))
    print("F1 Score:", f1_score(y_test, y_test_pred))
    print("\n")


Logistic Regression Training Metrics:
Accuracy: 0.91145625
Precision: 0.9157405536472014
Recall: 0.9063206969390178
F1 Score: 0.9110062754803978

Logistic Regression Testing Metrics:
Accuracy: 0.897825
Precision: 0.904485810192249
Recall: 0.8895113289651378
F1 Score: 0.8969360736351027


Naive Bayes Training Metrics:
Accuracy: 0.87785
Precision: 0.8926627787663494
Recall: 0.8590123364205632
F1 Score: 0.8755143377622645

Naive Bayes Testing Metrics:
Accuracy: 0.862775
Precision: 0.8777083333333333
Recall: 0.8428950132546391
F1 Score: 0.8599494807746281




In [23]:
# 7. SVM MODEL

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize the SVM model
model = SVC()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the labels for the training and testing data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate and print the metrics for the training data
print('Training data:')
print('Accuracy:', accuracy_score(y_train, y_train_pred))
print('Precision:', precision_score(y_train, y_train_pred))
print('Recall:', recall_score(y_train, y_train_pred))
print('F1 score:', f1_score(y_train, y_train_pred))

# Calculate and print the metrics for the testing data
print('Testing data:')
print('Accuracy:', accuracy_score(y_test, y_test_pred))
print('Precision:', precision_score(y_test, y_test_pred))
print('Recall:', recall_score(y_test, y_test_pred))
print('F1 score:', f1_score(y_test, y_test_pred))

Training data:
Accuracy: 0.97333125
Precision: 0.9730202348238821
Recall: 0.9736648043296211
F1 score: 0.973342412864613
Testing data:
Accuracy: 0.907
Precision: 0.9110796746324458
Recall: 0.9019656879907968
F1 score: 0.9064997737897753
